In [1]:
!pip install Mlxtend

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train.tail()

,Unnamed: 0,Time,data,label
20110,48,[03:36.79,It is meaningful,Sad
20111,49,[03:41.90,It is wonderful,Sad
20112,50,[03:46.64,It is meaningful,Sad
20113,51,[03:51.28,It goes full circle,Sad
20114,52,[03:58.30,NaN,Sad


In [3]:
df_train = df_train.dropna(how = 'any')
df_test = df_test.dropna(how = 'any')
print(df_train.tail(100))
print(df_test.tail(100))

       Unnamed: 0       Time                                         data  \
19993          75  [02:30.76  You won't find me, the past is so behind me   
19995          77  [02:37.64                           Buried in the snow   
19996          78  [02:42.34                                            .   
19998          80  [02:43.40                                    Let it go   
19999          81  [02:44.90                                    Let it go   
...           ...        ...                                          ...   
20109          47  [03:31.95                           It is so wonderful   
20110          48  [03:36.79                             It is meaningful   
20111          49  [03:41.90                              It is wonderful   
20112          50  [03:46.64                             It is meaningful   
20113          51  [03:51.28                          It goes full circle   

      label  
19993   Sad  
19995   Sad  
19996   Sad  
19998   Sad  
19999

In [4]:
df_train = df_train[~df_train['data'].str.startswith('[','.')]
print(df_train)
#df_test = df_test[~df_test['data'].str.startswith('[','.')]
print(df_test)

       Unnamed: 0       Time                            data  label
0               0  [00:12.55  Another day wasted out of time  Angry
1               1  [00:14.69         I can't get out of this  Angry
2               2  [00:16.03           Altered state of mind  Angry
3               3  [00:17.41             I'm going overboard  Angry
4               4  [00:18.75     My conscience meets decline  Angry
...           ...        ...                             ...    ...
20109          47  [03:31.95              It is so wonderful    Sad
20110          48  [03:36.79                It is meaningful    Sad
20111          49  [03:41.90                 It is wonderful    Sad
20112          50  [03:46.64                It is meaningful    Sad
20113          51  [03:51.28             It goes full circle    Sad

[17622 rows x 4 columns]
       Unnamed: 0       Time  \
0               0  [00:34.61   
1               1  [00:39.10   
2               2  [00:43.86   
3               3  [00:47.26  

In [5]:
X_train = df_train['data']
y_train = df_train['label']

X_test = df_test['data']
y_test = df_test['label']

In [6]:
from sklearn.preprocessing import LabelEncoder
import random
import pickle
import numpy as np

X_train = df_train['data'].values 

y_train = df_train['label'].values

print('before: %s ...' %y_train[:5])

le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

print('after: %s ...' %y_train[:5])

X_test = df_test['data'].values 

y_test = df_test['label'].values

print('before: %s ...' %y_test[:5])

le = LabelEncoder()
le.fit(y_test)
y_test = le.transform(y_test)

print('after: %s ...' %y_test[:5])

before: ['Angry' 'Angry' 'Angry' 'Angry' 'Angry'] ...
after: [0 0 0 0 0] ...
before: ['Angry' 'Angry' 'Angry' 'Angry' 'Angry'] ...
after: [0 0 0 0 0] ...


In [7]:
import pickle
stop_words = pickle.load(open('./stopwords.p', 'rb'))
semantic_words = pickle.load(open('./semantic_words_py34.p', 'rb'))

In [8]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import EnglishStemmer

porter = PorterStemmer()
snowball = EnglishStemmer()

# raw words
tokenizer = lambda text: text.split()

# words after Porter stemming 
tokenizer_porter = lambda text: [porter.stem(word) for word in text.split()]

# Words after Snowball stemming
tokenizer_snowball = lambda text: [snowball.stem(word) for word in text.split()]

# Only words that are in a list of 'positive' or 'negative' words ('whitelist')
# http://www.cs.uic.edu/~liub/FBS/sentiment-analysis.html#lexicon
tokenizer_whitelist = lambda text: [word for word in text.split() if word in semantic_words]

# Porter-stemmed words in whitelist
tokenizer_porter_wl = lambda text: [porter.stem(word) for word in text.split() if word in semantic_words]

# Snowball-stemmed words in whitelist
tokenizer_snowball_wl = lambda text: [snowball.stem(word) for word in text.split() if word in semantic_words]

In [9]:
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from mlxtend.preprocessing import DenseTransformer

vect_1 = CountVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer)

vect_2 = CountVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer_porter)
    
vect_3 = CountVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer_snowball)  

vect_4 = CountVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer_whitelist)  
vect_5 = CountVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer_porter_wl)

vect_6 = CountVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer_snowball_wl)

vect_7 = TfidfVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer)

vect_8 = TfidfVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer_porter)
    
vect_9 = TfidfVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer_snowball)

vect_10 = TfidfVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer_whitelist)    
vect_11 = TfidfVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer_porter_wl)

vect_12 = TfidfVectorizer(binary=False,
                         stop_words=stop_words,
                         ngram_range=(1,1),
                         preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                         tokenizer=tokenizer_snowball_wl)


pipelines = []
vectorizers = [vect_1, vect_2, vect_3, vect_4, vect_5, vect_6, vect_7, vect_8, vect_9, vect_10, vect_11, vect_12]
for v in vectorizers:
    pipelines.append(Pipeline([('vect', v),
                               ('dense', DenseTransformer()),
                               ('clf', RandomForestClassifier(n_estimators=100))]))

In [10]:
print('Vocabulary sizes\n')
labels = ['CountVec', 'CountVec porter', 'CountVec snowball', 'CountVec wl', 'CountVec porter+wl','CountVec snowball+wl',
          'TfidfVec', 'TfidfVec porter', 'TfidfVec snowball', 'TfidfVec wl', 'TfidfVec porter+wl','TfidfVec snowball+wl',]
X_train = df_train['data'].values.astype('U')
#print(X_train)
for label, v in zip(labels, vectorizers):
    v.fit(X_train)
    print('%s: %s' % (label, len(v.vocabulary_)))

Vocabulary sizes



/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVec: 5788


/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(


CountVec porter: 4474


/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'onc', 'onli', 'ourselv', 'themselv', 'veri', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(


CountVec snowball: 4457
CountVec wl: 1013
CountVec porter+wl: 831
CountVec snowball+wl: 824
TfidfVec: 5788
TfidfVec porter: 4474
TfidfVec snowball: 4457
TfidfVec wl: 1013
TfidfVec porter+wl: 831
TfidfVec snowball+wl: 824


In [11]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

labels = ['CountVec', 'CountVec porter', 'CountVec snowball', 'CountVec wl', 'CountVec porter+wl','CountVec snowball+wl',
          'TfidfVec', 'TfidfVec porter', 'TfidfVec snowball', 'TfidfVec wl', 'TfidfVec porter+wl','TfidfVec snowball+wl',]



d = {'Data':labels,
     'Accuracy (%)':[],}

for i,clf in enumerate(pipelines):
    #if i >= 6:
      #break
    scores = cross_val_score(estimator=clf, X=X_train, y=y_train, scoring='accuracy', cv=5)
    print('clf %s, %s: %s' % (i+1, labels[i], scores.mean()*100))
    d['Accuracy (%)'].append('%0.2f (+/- %0.2f)' % (scores.mean()*100, scores.std()*100))

clf 1, CountVec: 30.71733764822373


/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi'

clf 2, CountVec porter: 31.341563825762154


/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'onc', 'onli', 'ourselv', 'themselv', 'veri', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'onc', 'onli', 'ourselv', 'themselv', 'veri', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'onc', 'onli', 'ourselv', 'themselv', 'veri', 'whi', 'yourse

clf 3, CountVec snowball: 31.080538717286128
clf 4, CountVec wl: 30.881934616530216
clf 5, CountVec porter+wl: 30.541465613704606
clf 6, CountVec snowball+wl: 30.717365018797143
clf 7, TfidfVec: 32.92475346358506


/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi'

clf 8, TfidfVec porter: 33.57741605686639


/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'onc', 'onli', 'ourselv', 'themselv', 'veri', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'onc', 'onli', 'ourselv', 'themselv', 'veri', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'onc', 'onli', 'ourselv', 'themselv', 'veri', 'whi', 'yourse

clf 9, TfidfVec snowball: 33.236898753028875
clf 10, TfidfVec wl: 31.36426852142552
clf 11, TfidfVec porter+wl: 30.93864966471048
clf 12, TfidfVec snowball+wl: 30.904636092126132


In [12]:
df_perform = pd.DataFrame(d)
df_perform = df_perform['Accuracy (%)']
df_perform.index=(labels)
df_perform

CountVec                30.72 (+/- 2.27)
CountVec porter         31.34 (+/- 2.93)
CountVec snowball       31.08 (+/- 2.30)
CountVec wl             30.88 (+/- 0.66)
CountVec porter+wl      30.54 (+/- 0.86)
CountVec snowball+wl    30.72 (+/- 0.86)
TfidfVec                32.92 (+/- 1.92)
TfidfVec porter         33.58 (+/- 2.59)
TfidfVec snowball       33.24 (+/- 2.01)
TfidfVec wl             31.36 (+/- 0.84)
TfidfVec porter+wl      30.94 (+/- 0.76)
TfidfVec snowball+wl    30.90 (+/- 1.04)
Name: Accuracy (%), dtype: object

In [13]:
vect = TfidfVectorizer(binary=False,
                       stop_words=stop_words,
                       ngram_range=(1,1),
                       preprocessor=lambda text: re.sub('[^a-zA-Z]', ' ', text.lower()),
                       tokenizer=lambda text: [porter.stem(word) for word in text.split()])

In [14]:
from sklearn.model_selection import cross_val_score

pipe_1 = Pipeline([
                ('vect',   vect),
                ('dense', DenseTransformer()),
                ('clf', RandomForestClassifier(n_estimators=50))])

pipe_2 = Pipeline([
                ('vect',   vect),
                ('dense', DenseTransformer()),
                ('clf', RandomForestClassifier(n_estimators=100))])

pipe_3 = Pipeline([
                ('vect',   vect),
                ('dense', DenseTransformer()),
                ('clf', RandomForestClassifier(n_estimators=200))])

pipe_4 = Pipeline([
                ('vect',   vect),
                ('dense', DenseTransformer()),
                ('clf', RandomForestClassifier(n_estimators=400))])

labels = [50, 100, 200, 400]

for i,clf in enumerate([pipe_1, pipe_2, pipe_3, pipe_4]):
    scores = cross_val_score(estimator=clf, X=X_train, y=y_train, scoring='accuracy', cv=5)
    print('clf %s, %s: %0.2f (+/- %0.2f)' % (i+1, labels[i], scores.mean()*100, scores.std()*100))

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi'

clf 1, 50: 32.83 (+/- 2.48)


/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi'

clf 2, 100: 33.17 (+/- 2.14)


/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi'

clf 3, 200: 33.17 (+/- 2.09)


/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi'

clf 4, 400: 33.33 (+/- 2.10)


In [15]:
X_train_feat = vect.fit_transform(X_train, y_train)
X_train_feat = X_train_feat.toarray()

print(X_train_feat)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  warnings.warn(


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pandas as pd

clf_2 = RandomForestClassifier(n_estimators=100)


tuned_parameters = [
  {'criterion': ['gini', 'entropy'], 
   'max_features': ['auto', 'log2', 'sqrt'],
   'min_samples_split':[2,3], 
   'min_samples_leaf':[1,2]},
 ]


grid_search_1 = GridSearchCV(clf_2, 
                           tuned_parameters, 
                           n_jobs=1, 
                           scoring='accuracy',
                           cv=5
                )

grid_search_1.fit(X_train_feat, y_train)

df_gs = pd.DataFrame(grid_search_1.cv_results_)
print(df_gs)

print("Best parameters set found on development set:")
print()
print(grid_search_1.best_estimator_)
print()
print("Grid scores on development set:")
print()

#for params, mean_score, scores in df_gv:
    #print("%0.3f (+/-%0.03f) for %r"
            #% (mean_score, scores.std() / 2, params))

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0      167.510064      6.054650         1.187004        0.070274   
1      163.459091      4.682669         1.140103        0.035122   
2      142.986353      2.819247         0.929640        0.051935   
3      144.480574      4.714326         0.924442        0.015381   
4       65.799285      1.385040         1.405238        0.021545   
5       62.795057      1.001090         1.399284        0.029284   
6       20.170483      0.068209         0.519298        0.010660   
7       20.780796      1.067379         0.528959        0.024659   
8      164.500692      5.269845         1.138344        0.023134   
9      162.918033      4.017413         1.131404        0.044386   
10     142.933088      4.553183         0.915286        0.017867   
11     150.396967      3.755187         0.960262        0.035249   
12     153.676237      3.535345         1.128734        0.167482   
13     152.701891      3.750784         1.091205

In [18]:
print(df_gs)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0      167.510064      6.054650         1.187004        0.070274   
1      163.459091      4.682669         1.140103        0.035122   
2      142.986353      2.819247         0.929640        0.051935   
3      144.480574      4.714326         0.924442        0.015381   
4       65.799285      1.385040         1.405238        0.021545   
5       62.795057      1.001090         1.399284        0.029284   
6       20.170483      0.068209         0.519298        0.010660   
7       20.780796      1.067379         0.528959        0.024659   
8      164.500692      5.269845         1.138344        0.023134   
9      162.918033      4.017413         1.131404        0.044386   
10     142.933088      4.553183         0.915286        0.017867   
11     150.396967      3.755187         0.960262        0.035249   
12     153.676237      3.535345         1.128734        0.167482   
13     152.701891      3.750784         1.091205

In [19]:
from sklearn import metrics

acc_scorer = metrics.make_scorer(metrics.accuracy_score, greater_is_better=True)
pre_scorer = metrics.make_scorer(metrics.precision_score, greater_is_better=True, average = 'macro')
rec_scorer = metrics.make_scorer(metrics.recall_score, greater_is_better=True, average = 'macro')
f1_scorer = metrics.make_scorer(metrics.f1_score, greater_is_better=True, average = 'macro')

In [21]:
labels = ['Train CountVec', 'Train CountVec porter', 'Train CountVec snowball', 'Train CountVec wl', 
          'Train CountVec porter+wl','Train CountVec snowball+wl',
          'Train TfidfVec', 'Train TfidfVec porter', 'Train TfidfVec snowball', 'Train TfidfVec wl', 
          'Train TfidfVec porter+wl','Train TfidfVec snowball+wl',
          'Test CountVec', 'Test CountVec porter', 'Test CountVec snowball', 'Test CountVec wl', 
          'Test CountVec porter+wl','Test CountVec snowball+wl',
          'Test TfidfVec', 'Test TfidfVec porter', 'Test TfidfVec snowball', 'Test TfidfVec wl', 
          'Test TfidfVec porter+wl','Test TfidfVec snowball+wl',]

d = {'Data':labels,
     'ACC (%)':[],
     'PRE (%)':[],
     'REC (%)':[],
     'F1 (%)':[],
}

for clf in pipelines:
    clf.fit(X_train, y_train)

for clf in pipelines:

    d['ACC (%)'].append(acc_scorer(estimator=clf, X=X_train, y_true=y_train))
    d['PRE (%)'].append(pre_scorer(estimator=clf, X=X_train, y_true=y_train))
    d['REC (%)'].append(rec_scorer(estimator=clf, X=X_train, y_true=y_train))
    d['F1 (%)'].append(f1_scorer(estimator=clf, X=X_train, y_true=y_train))

for clf in pipelines:

    d['ACC (%)'].append(acc_scorer(estimator=clf, X=X_test, y_true=y_test))
    d['PRE (%)'].append(pre_scorer(estimator=clf, X=X_test, y_true=y_test))
    d['REC (%)'].append(rec_scorer(estimator=clf, X=X_test, y_true=y_test))
    d['F1 (%)'].append(f1_scorer(estimator=clf, X=X_test, y_true=y_test))

In [22]:
print(d)
print(len(d['Data']))
print(len(d['ACC (%)']))
print(len(d['PRE (%)']))
print(len(d['REC (%)']))
print(len(d['F1 (%)']))

{'Data': ['Train CountVec', 'Train CountVec porter', 'Train CountVec snowball', 'Train CountVec wl', 'Train CountVec porter+wl', 'Train CountVec snowball+wl', 'Train TfidfVec', 'Train TfidfVec porter', 'Train TfidfVec snowball', 'Train TfidfVec wl', 'Train TfidfVec porter+wl', 'Train TfidfVec snowball+wl', 'Test CountVec', 'Test CountVec porter', 'Test CountVec snowball', 'Test CountVec wl', 'Test CountVec porter+wl', 'Test CountVec snowball+wl', 'Test TfidfVec', 'Test TfidfVec porter', 'Test TfidfVec snowball', 'Test TfidfVec wl', 'Test TfidfVec porter+wl', 'Test TfidfVec snowball+wl'], 'ACC (%)': [0.9435364884803087, 0.9447849279309953, 0.9423447962773805, 0.4288956985586199, 0.41936216093519463, 0.41919191919191917, 0.9402451481103167, 0.9409261150834185, 0.9385994779253206, 0.4261150834184542, 0.4156168425831347, 0.41550334808761774, 0.29160475482912335, 0.2920381376919267, 0.29240960871718674, 0.2638682516097078, 0.2642397226349678, 0.26492075284794453, 0.3070208023774146, 0.31785

In [23]:
pd.set_option('precision', 2)

df_perform = pd.DataFrame(d)
df_perform = df_perform[['ACC (%)', 'PRE (%)', 'REC (%)', 'F1 (%)']]
df_perform.index=(labels)
df_perform = df_perform*100
df_perform = np.round(df_perform, decimals=2)
df_perform

,ACC (%),PRE (%),REC (%),F1 (%)
Train CountVec,94.35,94.35,94.09,94.20
Train CountVec porter,94.48,94.31,94.28,94.29
Train CountVec snowball,94.23,94.09,94.03,94.04
Train CountVec wl,42.89,54.72,39.80,38.15
Train CountVec porter+wl,41.94,53.10,38.81,36.82
Train CountVec snowball+wl,41.92,53.41,38.78,36.74
Train TfidfVec,94.02,94.02,93.75,93.87
Train TfidfVec porter,94.09,93.90,93.91,93.89
Train TfidfVec snowball,93.86,93.73,93.66,93.67
Train TfidfVec wl,42.61,54.25,39.56,37.81
